First import required packages

In [ ]:
import pandas as pd
import requests as rq
import subprocess as sp
import numpy as np
import panel as pn
pn.extension('tabulator')

import hvplot.pandas

#pip install ipykernel
#pip install jupyter_bokeh
#pip install nbconvert
#https://panel.holoviz.org/api/panel.widgets.html

Next get data from ONS
Get link from: https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathsregisteredinenglandandwalesseriesdrreferencetables
xls = pd.ExcelFile('https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathsregisteredinenglandandwalesseriesdrreferencetables/2022/dr2022corrected.xlsx')

The url doesn't accept "non-browser" requests. The default header of Python requests is 'User-Agent': 'python-requests/2.13.0', need to pass different headers as an argument
https://stackoverflow.com/questions/59297759/python-requests-module-to-download-an-excel-file-from-a-download-link-which-po

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
response = requests.get('https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/deathsregisteredinenglandandwalesseriesdrreferencetables/2022/dr2022corrected.xlsx', headers=headers, verify=False)
with open("DeathsData.xlsx", 'wb') as f:
    f.write(response.content)

# Bring in a tab of the xlsx as a data frame
df = pd.read_excel('DeathsPub.xlsx', sheet_name='2b', skiprows=4)
print(df)

Next get data from the Lake - note this will only work if you are on the network and have read access to the table and views in the query. Changes to the query can be performed by editing 'GetLakeDeaths.SQL'. Specify the path to your batch file and run the batch file using subprocess.

In [ ]:
batch_file_path = 'GetLakeDeaths.bat'
subprocess.call([batch_file_path])

Join the datasets together